In [1]:
import numpy as np

from s4_1_sensible_heat import get_load_and_temp

未知のパラメータ（室温・対流負荷・放射負荷）が室ごとに変わる場合にそれぞれを求める関数が正しいかをチェックする

室温・対流負荷・放射負荷は下式で表されると仮定する。

$$ \boldsymbol{kt} \: \boldsymbol{\theta} = \boldsymbol{kc} \: \boldsymbol{Lc} + \boldsymbol{kr} \: \boldsymbol{Lr} + \boldsymbol{k} $$

ここで検証用として、kt, kc, kr, k の値として以下の値とする。（部屋数は3部屋を想定する。）

In [2]:
kt = np.array([[1.0, 0.1, 0.1],[0.1, 1.0, 0.1],[0.1, 0.1, 1.0]])
kc = np.array([[6.0, 1.0, 0.5],[2.5, 8.0, 1.5],[3.0, 2.5, 8.0]])
kr = np.array([[4.0, 2.0, 1.0],[2.5, 4.0, 2.0],[1.0, 0.5, 6.0]])
k = np.array([[20.0],[18.0],[10.0]])

テスト用の関数

In [3]:
def for_test(_T, _LC, _LR):

    def cvt(d):
        
        p1 = np.zeros((3,1))
        p2 = np.zeros((3,1))
        
        for i in range(3):
            if d[i] == 'ND':
                p1[i, 0] = 1
                p2[i, 0] = 0.0
            else:
                p1[i, 0] = 0
                p2[i, 0] = float(d[i])
        
        return p1, p2
                
    nt, theta_set = cvt(_T)

    c, lc_set = cvt(_LC)
    
    r, lr_set = cvt(_LR)

    theta, lc, lr = get_load_and_temp(
        kt=kt, kc=kc, kr=kr, k=k, nt=nt, theta_set=theta_set, c=c, lc_set=lc_set, r=r, lr_set=lr_set)

    print('===計算された値===')
    print('室温: [0] ' + str(theta[0, 0]) + ' [1] ' + str(theta[1, 0]) + ' [2] '+ str(theta[2, 0]))
    print('対流負荷 [0] ' + str(lc[0, 0]) + ' [1] ' + str(lc[1, 0]) + ' [2] '+ str(lc[2, 0]))
    print('放射負荷 [0] ' + str(lr[0, 0]) + ' [1] ' + str(lr[1, 0]) + ' [2] '+ str(lr[2, 0]))
    
    print('===設置が維持されていることを確認===')
    theta_set[nt == 1] = np.nan
    print('設定された温度: [0] '
          + str(theta_set[0, 0]) + ' [1] ' + str(theta_set[1, 0]) + ' [2] '+ str(theta_set[2, 0]))
    lc_set[c == 1] = np.nan
    print('設定された対流負荷: [0] '
          + str(lc_set[0, 0]) + ' [1] ' + str(lc_set[1, 0]) + ' [2] '+ str(lc_set[2, 0]))
    lr_set[r == 1] = np.nan
    print('設定された放射負荷: [0] '
          + str(lr_set[0, 0]) + ' [1] ' + str(lr_set[1, 0]) + ' [2] '+ str(lr_set[2, 0]))

    print('===Lc Lr から theta を計算し、一致していることを確認する。===')
    tc = np.dot(np.linalg.inv(kt), np.dot(kc, lc) + np.dot(kr, lr) + k)
    print('再計算された室温（検算用）: [0] ' + str(tc[0, 0]) + ' [1] ' + str(tc[1, 0]) + ' [2] '+ str(tc[2, 0]))

# 検算

空調をしない場合  
- 室温を指定しない  
- 対流空調負荷は0を指定する  
- 放射空調負荷は0を指定する

In [4]:
for_test(_T=['ND', 'ND', 'ND'], _LC=['0.0', '0.0', '0.0'], _LR=['0.0', '0.0', '0.0'])

===計算された値===
室温: [0] 17.77777777777778 [1] 15.555555555555559 [2] 6.666666666666668
対流負荷 [0] 0.0 [1] 0.0 [2] 0.0
放射負荷 [0] 0.0 [1] 0.0 [2] 0.0
===設置が維持されていることを確認===
設定された温度: [0] nan [1] nan [2] nan
設定された対流負荷: [0] 0.0 [1] 0.0 [2] 0.0
設定された放射負荷: [0] 0.0 [1] 0.0 [2] 0.0
===Lc Lr から theta を計算し、一致していることを確認する。===
再計算された室温（検算用）: [0] 17.77777777777778 [1] 15.555555555555559 [2] 6.666666666666668


対流空調をする場合  
- 室温を指定する（20.0, 20.0, 20.0 とする。）
- 対流空調負荷は指定しない
- 放射空調負荷

In [5]:
for_test(_T=['20.0', '20.0', '20.0'], _LC=['ND', 'ND', 'ND'], _LR=['0.0', '0.0', '0.0'])

===計算された値===
室温: [0] 20.0 [1] 20.0 [2] 20.0
対流負荷 [0] 0.49091583240637743 [1] 0.3218390804597701 [2] 1.4653318502039303
放射負荷 [0] 0.0 [1] 0.0 [2] 0.0
===設置が維持されていることを確認===
設定された温度: [0] 20.0 [1] 20.0 [2] 20.0
設定された対流負荷: [0] nan [1] nan [2] nan
設定された放射負荷: [0] 0.0 [1] 0.0 [2] 0.0
===Lc Lr から theta を計算し、一致していることを確認する。===
再計算された室温（検算用）: [0] 20.000000000000004 [1] 20.000000000000004 [2] 20.0


放射空調をする場合  
- 室温を指定する（20.0, 20.0, 20.0 とする。）
- 対流空調負荷はは0.0とする。
- 放射空調負荷は指定しない。

In [6]:
for_test(_T=['20.0', '20.0', '20.0'], _LC=['0.0', '0.0', '0.0'], _LR=['ND', 'ND', 'ND'])

===計算された値===
室温: [0] 20.0 [1] 20.0 [2] 20.0
対流負荷 [0] 0.0 [1] 0.0 [2] 0.0
放射負荷 [0] 0.36363636363636376 [1] 0.14229249011857692 [2] 2.2608695652173916
===設置が維持されていることを確認===
設定された温度: [0] 20.0 [1] 20.0 [2] 20.0
設定された対流負荷: [0] 0.0 [1] 0.0 [2] 0.0
設定された放射負荷: [0] nan [1] nan [2] nan
===Lc Lr から theta を計算し、一致していることを確認する。===
再計算された室温（検算用）: [0] 20.000000000000004 [1] 20.000000000000004 [2] 20.0


放射空調に上限値が指定され、その残りを対流空調で補う場合  
- 室温を指定する
- 対流空調負荷は指定しない
- 放射空調負荷は指定する（1つ前の値を参考に放射空調だけのパターン

In [7]:
for_test(_T=['20.0', '20.0', '20.0'], _LC=['ND', 'ND', 'ND'], _LR=['0.2', '0.05', '2.0'])

===計算された値===
室温: [0] 20.0 [1] 20.0 [2] 20.0
対流負荷 [0] 0.1573971078976643 [1] 0.08793103448275866 [2] 0.13537263626251383
放射負荷 [0] 0.2 [1] 0.05 [2] 2.0
===設置が維持されていることを確認===
設定された温度: [0] 20.0 [1] 20.0 [2] 20.0
設定された対流負荷: [0] nan [1] nan [2] nan
設定された放射負荷: [0] 0.2 [1] 0.05 [2] 2.0
===Lc Lr から theta を計算し、一致していることを確認する。===
再計算された室温（検算用）: [0] 20.000000000000004 [1] 20.000000000000004 [2] 20.0


空調を最大能力でするが室温が所定の設定温度を満たさない場合  
- 室温を指定しない  
- 対流空調負荷は最大能力を指定する  
- 放射空調負荷は0を指定する

In [8]:
for_test(_T=['ND', 'ND', 'ND'], _LC=['0.4', '0.3', '1.3'], _LR=['0.0', '0.0', '0.0'])

===計算された値===
室温: [0] 19.55092592592593 [1] 19.550925925925927 [2] 18.439814814814817
対流負荷 [0] 0.4 [1] 0.3 [2] 1.3
放射負荷 [0] 0.0 [1] 0.0 [2] 0.0
===設置が維持されていることを確認===
設定された温度: [0] nan [1] nan [2] nan
設定された対流負荷: [0] 0.4 [1] 0.3 [2] 1.3
設定された放射負荷: [0] 0.0 [1] 0.0 [2] 0.0
===Lc Lr から theta を計算し、一致していることを確認する。===
再計算された室温（検算用）: [0] 19.55092592592593 [1] 19.550925925925927 [2] 18.439814814814817
